# Queues
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/matyama/pfds/blob/main/notebooks/queues.ipynb)

|       instance      | persistence | amortization | empty | isEmpty |      snoc     | head |      tail     |
|:-------------------:|:-----------:|:------------:|:-----:|:-------:|:-------------:|:----:|:-------------:|
|    Batched Queue    |  ephemeral  |      yes     |  O(1) |   O(1)  |  O(n) / O(1)* | O(1) |  O(n) / O(1)* |
|    Banker's Queue   |  persistent |      yes     |  O(1) |   O(1)  |  O(n) / O(1)* | O(1) |  O(n) / O(1)* |
|  Physicist's Queue  |  persistent |      yes     |  O(1) |   O(1)  |  O(n) / O(1)* | O(1) |  O(n) / O(1)* |
|   Real-Time Queue   |  persistent |      no      |  O(1) |   O(1)  |      O(1)     | O(1) |      O(1)     |
| Hood-Melville Queue |  persistent |      no      |  O(1) |   O(1)  |      O(1)     | O(1) |      O(1)     |
|  Bootstrapped Queue |  persistent |      yes     |  O(1) |   O(1)  | O(log\*(n))** | O(1) | O(log\*(n))** |

** amortized time*

*** amortized time, $log^*(n)$ is constant in practice*

In [ ]:
class Queue q where
    
    -- | Construct new (empty) queue
    empty :: q a
    
    -- | Check whether a queue is empty
    isEmpty :: q a -> Bool
    
    -- | Append new item to the back of a queue
    snoc :: q a -> a -> q a
    
    -- | Retrieve the head item of a non-empty queue
    head :: q a -> a
    
    -- | Remove the head of a non-empty queue and retrieve the rear
    tail :: q a -> q a

## Batched Queue

In [ ]:
data BatchedQueue a = BQ [a] [a]

-- | Helper function that maintains the 'BatchedQueue' invariant:
-- | *A queue is empty iff the front part is empty*
-- | 
-- | This invariant is preserved by reversing the rear and replacing the front
-- | whenever the front is empty.
checkf :: ([a], [a]) -> ([a], [a])
checkf ([], r) = (reverse r, [])
checkf q = q

instance Queue BatchedQueue where
    
    -- | Constructs new queue in O(1)
    empty = BQ [] []

    -- | Checks whether the queue is empty in O(1) steps.
    -- | Note: This queue maintains the invariant that if the front part is empty, the queue is empty
    -- |       check the 'checkf' helper function.
    isEmpty (BQ f _) = null f
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (BQ f r) x = let (f', r') = checkf(f, x : r) in BQ f' r'
    
    -- | Retrieves the head of the queue in O(1)
    head (BQ [] _) = error "Queue is empty"
    head (BQ (x:_) _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (BQ [] _) = error "Queue is empty"
    tail (BQ (x:f) r) = let (f', r') = checkf (f, r) in BQ f' r'

## Banker's Queue
`BankersQueue` is a persistent version of the `BatchedQueue` that is based on the *banker's method*.

In [ ]:
data BankersQueue a = BQ Int [a] Int [a]

-- | Rotates the queue if |r| > |f|.
check :: Int -> [a] -> Int -> [a] -> BankersQueue a
check lenf f lenr r =
    if lenr <= lenf then BQ lenf f lenr r
    else BQ (lenf + lenr) (f ++ reverse r) 0 []

instance Queue BankersQueue where

    -- | Constructs new queue in O(1)
    empty = BQ 0 [] 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (BQ lenf _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (BQ lenf f lenr r) x = check lenf f (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1)
    head (BQ _ [] _ _) = error "Queue is empty"
    head (BQ _ (x:_) _ _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (BQ _ [] _ _) = error "Queue is empty"
    tail (BQ lenf (x:f) lenr r) = check (lenf - 1) f lenr r

## Physicist's Queue
`PhysicistsQueue` is a persistent version of the `BatchedQueue` that is based on the *physicist's method*.

In [ ]:
data PhysicistsQueue a = PQ [a] Int [a] Int [a]

-- | Rotates the queue if |r| > |f| and constructs the queue via 'checkw'.
check :: [a] -> Int -> [a] -> Int -> [a] -> PhysicistsQueue a
check w lenf f lenr r =
    if lenr <= lenf then checkw w lenf f lenr r
    else checkw f (lenf + lenr) (f ++ reverse r) 0 []

-- | Checks the and possibly updates the front prefix (the working list).
checkw :: [a] -> Int -> [a] -> Int -> [a] -> PhysicistsQueue a
checkw [] lenf f lenr r = PQ f lenf f lenr r
checkw w lenf f lenr r = PQ w lenf f lenr r

instance Queue PhysicistsQueue where
    -- | Constructs new queue in O(1)
    empty = PQ [] 0 [] 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (PQ _ lenf _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (PQ w lenf f lenr r) x = check w lenf f (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1)
    head (PQ [] _ _ _ _) = error "Queue is empty"
    head (PQ (x:_) _ _ _ _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (PQ [] _ _ _ _) = error "Queue is empty"
    tail (PQ (x:w) lenf f lenr r) = check w (lenf - 1) (Prelude.tail f) lenr r

## Real-Time Queue
`RealTimeQueue` eliminates the amortization of the other persistent queue instances and thus has all operations $O(1)$ in the worst case.

This queue implementation incorporates *lazy rebuilding* to make rotation incremental and *scheduling* to ensure the efficiency of rotation (see below) and thus is an instance of a *global rebuilding* data structure.

It is the simplest and fastest queue implementation for applications utilizing persistence.

### Representation
The queue data structure has three components
 1. the front of the queue
 1. the (reversed) rear of the queue
 1. a *schedule* containing suspensions constructing nodes of the front that will be forced at specific (scheduled) time later on

### Rotation
The idea here is to break the monolithic rotation of items from the rear list to the front and make it incremental. 

The rotation of a `BankersQueue` consists of two operations: `(++)` and `reverse`. To make the rotation incremental we execute one step of the reversal for every step of `++`. Our new `rotate` function has an extra *accumulating parameter* which is used to accumulate the partial results of `reverse`.

The rotation occurs when $|r| = |f| + 1$ - this is maintained as an invariant.

### Scheduling
Members of queue's *schedule* are carefully evaluated via `exec` so that all the front dependencies (the `xs` list) are forced (and thus memoized) prior to the call to `rotate` for respective front list. This in turn make the `rotate` run in constant time, creating just suspensions.

In [ ]:
-- | RTQ = (front, rear, schedule)
data RealTimeQueue a = RTQ [a] [a] [a]

-- | Lazily rotate the rear to the front via scheduled suspension.
-- |
-- | This function performs both '(f++)' and 'reverse r' incrementally and
-- | in a lazy fastion. The treatment of schedule guarantees that every node
-- | in 'xs' was forced and memoized prior to the rotation.
-- |
-- | Hence the O(1) execution of every suspension in 'exec'.
rotate :: [a] -> [a] -> [a] -> [a]
rotate [] (y:_) a = y:a
rotate (x:xs) (y:ys) a = x : rotate xs ys (y:a)

-- | Executes a suspension from the schedule.
-- |
-- | When the schedule is non-empty we just pop out its head.
-- |
-- | Otherwise we rotate the queue to produce new front and yield new queue
-- | with it as new front and schedule.
-- |
-- | Since schedule equals new front after a rotation and the next rotation
-- | happens when we force all suspensions from the schedule, 'exec' maintains
-- | the invariant that |s| = |f| - |r| which corresponds to |f| >= |r|.
-- |
-- | Note: Pattern matching against 's' forces and memoizes the next suspension.
exec :: [a] -> [a] -> [a] -> RealTimeQueue a
exec f r (_:s) = RTQ f r s
exec f r [] = let f' = rotate f r [] in RTQ f' [] f'

instance Queue RealTimeQueue where

    -- | Constructs new queue in O(1)
    empty = RTQ [] [] []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (RTQ [] _ _) = True
    isEmpty _ = False
    
    -- | Appends new element to the back of the queue in O(1) worst case time.
    -- |
    -- | New item is added to the rear and a suspension is executed from the schedule.
    snoc (RTQ f r s) x = exec f (x:r) s
    
    -- | Retrieves the head of the queue in O(1)
    head (RTQ [] _ _) = error "Queue is empty"
    head (RTQ (x:_) _ _) = x
    
    -- | Removes the head of the queue and returns the rest in O(1) worst case time.
    -- |
    -- | Head of the front is popped out and a suspension is executed from the schedule and
    -- | with the rest of the front list.
    tail (RTQ [] _ _) = error "Queue is empty"
    tail (RTQ (x:f) r s) = exec f r s

## Hood-Melville Real-Time Queue
`HoodMelvilleQueue` is a real-time queue similar to the `RealTimeQueue` (i.e. persistent with worst case constant operations).

The difference is that this instance is a closer representant of the *global rebuilding* technique while the `RealTimeQueue` showcases the *lazy rebuilding* and *scheduling* combination to achieve the same properties. Particularly, the differences lie in the details of the incremental rotation and consequently the queue representation.

### Rotation
Again the goal is to make reversal and then append plus reversal incremental.

First an incremental reversal can be captured by two states: `Working xs ys` and `Done xs` where the former represents gradual transfer of the head of `xs` to `ys` and the letter the final state.

Appending two lists `xs` and `ys` works similarly in two passes:
 1. reverse `xs` to get `xs'` - state `Reversing xs xs' ys`
 1. reverse `xs'` onto `ys` - state `Appending xs' ys`

Finally, to append `f` onto `reverse r` we need three reversals in total, two  of which can be done in parallel:
 1. reverse `f` and `r` in parallel to get `f'` and `r'` - state `Reversing f f' r r'`
 1. reverse `f'` onto `r'` - state `Appending f' r'`

There is, however, a problem with the state transitioning if done in a naive way. If we call `tail` during a rotation $k$ times then the first $k$ elements of the resulting list are invalid. To fix this we have few options:
 1. Track the number of invalid elements and add new rotational state `Deleting` that removes requred number of invalid elements (this corresponds to the definition of *global rebuilding*)
 1. Track the number `ok` of valid elements in `f'` and stop the rotation from `f'` ot `r'` when `ok == 0`. Every call to `tail` during a rotation then `invalidate`s elements (decrements `ok`).

### Representation
First the `RotationState` consists of four states:
 - `Reversing`, `Appending` and `Done` correspond to the states described above
 - `Idle` is a dummy state that is used for situations "inbetween" rotations

Then the queue itself is a five-tuple consisting of
 1. field `lenf` is the length of the list that is under construction (not the working copy - the front field)
 1. next field is a working copy of the front of the queue (an old copy of the front that is under construction to efficiently answer `head` queries)
 1. then there's the `RotationState` described above
 1. field `lenr` is the length of the rear
 1. the last field is the rear of the queue

During a rotation the head elements of the queue end up in the back of `f'` within the rotational state. By keeping a working copy of the old front in the queue we can efficiently answer `head` queries. This is sound as long as new copy is ready by the time the old is exhausted.

In [ ]:
data RotationState a = Idle -- | dummy state
                       -- | (no. valid, f, f', r, r') - moving f onto f' and r onto r'
                       | Reversing Int [a] [a] [a] [a]
                       -- | (no. valid, f', r') - moving f' onto r'
                       | Appending Int [a] [a]
                       -- | final result
                       | Done [a]

-- | HMQ = (len of f' in rotational state, old front, rotational state, len of rear, rear)
data HoodMelvilleQueue a = HMQ Int [a] (RotationState a) Int [a]

-- | Rotational state transition function (runs in constant time).
-- |  - Keep reversing until the front is exhausted (increment no. valid elements)
-- |  - Keep moving elements from f' to r' until f' is exhausted (decrements no. valid elements)
-- |  - Idle and Done states are terminal
exec :: RotationState a -> RotationState a
exec (Reversing ok (x:f) f' (y:r) r') = Reversing (ok + 1) f (x:f') r (y:r')
exec (Reversing ok [] f' [y] r') = Appending ok f' (y:r')
exec (Appending 0 _ r') = Done r'
exec (Appending ok (x:f') r') = Appending (ok - 1) f' (x:r')
exec state = state

-- | State transition function that removes invalid elements (runs in constant time).
-- |  - Decrements no. valid elements in Reversal state and Appending state with some valid elements
-- |  - Terminates appending state when there are no valid elements
-- |  - Noop for other states
invalidate :: RotationState a -> RotationState a
invalidate (Reversing ok f f' r r') = Reversing (ok - 1) f f' r r'
invalidate (Appending 0 _ (_:r')) = Done r'
invalidate (Appending ok f' r') = Appending (ok - 1) f' r'
invalidate state = state

-- | Make two rotational transitions and return resulting queue.
-- |
-- | Since 'exec' runs in O(1) worst case time so does 'exec'. Also by executing two transitions
-- | 'exec2' ensures that the rotation finishes in time to replace the working copy.
exec2 :: Int -> [a] -> RotationState a -> Int -> [a] -> HoodMelvilleQueue a
exec2 lenf f state lenr r = case exec $ exec state of
    (Done newf) -> HMQ lenf newf Idle lenr r
    newstate -> HMQ lenf f newstate lenr r

-- | Run partial rotation and return new queue.
-- |  - When |r| = |f| + 1 new rotation is started and partially executed via 'exec2'
-- |  - Otherwise we just run two rotational steps from the current state
-- |
-- | Runs in O(1) worst case for the same reason as 'exec2' does.
-- |
-- | Note: When new rotaiton starts, the resulting queue front is a workig copy of the old f.
check :: Int -> [a] -> RotationState a -> Int -> [a] -> HoodMelvilleQueue a
check lenf f state lenr r =
    if lenr <= lenf then exec2 lenf f state lenr r
    else let newstate = Reversing 0 f [] r [] in exec2 (lenf + lenr) f newstate 0 []

instance Queue HoodMelvilleQueue where

    -- | Constructs new queue in O(1)
    empty = HMQ 0 [] Idle 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (HMQ lenf _ _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue in O(1) worst case time.
    snoc (HMQ lenf f state lenr r) x = check lenf f state (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1) worst case time.
    -- |
    -- | Note: Since the front is a working copy of the old front we can just peek its head.
    -- |       Moreover, this is sound because 'snoc' and 'tail' ensure (via 'exec2') that the
    -- |       new copy replaces the front before it is ever exhausted.
    head (HMQ _ [] _ _ _) = error "Queue is empty"
    head (HMQ _ (x:_) _ _ _) = x
    
    -- | Removes the head of the queue and returns the rest in O(1) worst case time.
    -- |
    -- | Note: Calls to 'tail' must invalidate elements in case the state is an ongoing rotation.
    tail (HMQ _ [] _ _ _) = error "Queue is empty"
    tail (HMQ lenf (x:f) state lenr r) = check (lenf - 1) f (invalidate state) lenr r

## Bootstrapped Queue
*Bootstrapped queue* is a queue instance based on *structural decomposition* of the `BankersQueue`.

A series of rotations on `BankersQueue` results in structure that looks like `((f ++ reverse r1) ++ reverse r2) ++ ... ++ reverse rk`. It is obvious that `BankersQueue` processes the front $k$ times and each $r_i$ is accessed $k - i - 1$ times. The total cost is linear just because each $r_{i+1}$ is twice as long as $r_i$.

`BootstrappedQueue` eliminates most suspensions by explicitly modelling the lazy evaluation of the rotation via operations on a FIFO queue. The rotation decomposes into
 - operations on the front part `f`
 - operations on  the middle queue `m = [reverse r1, ..., reverse rk]`

### Representaion
`BootstrappedQueue` has two constructors: `E` represents an empty queue while `Q` constructs a non-empty queue as a five-tuple consisting of
 1. field `lenfm` is the combined length of the front and all the lists in the middle queue (see next fields)
 1. next is the front part of the queue that is assumed to be non-empty
 1. then there's the middle part which itself is a `BootstrappedQueue` but with elements being lists of elements `[a]` that represent reversals of previous rear lists
 1. field `lenr` is the length of the rear
 1. the last field is the rear of the queue

The front and middle part (queue) represent a decomposition of the `BankersQueue`.

### Operations
The analysis of `reverse` and it's impact on the amortized complexity of operations is similar to the `BankersQueue` (contributes $O(1)$ amortized time).

The difference is that `checkQ` may indirectly cascade reversals because it calls `snoc` on the middle queue. Fortunatly, the lists in the middle double in size and the size of the middle decreases by a logarithmic factor.

Therefore, `snoc` (and analogously `tail`) take $log^*(n)$ amortized time which is a constnat in practice.

### Benefits
> In practice, variations on these queues are the fastest known implementations for applications that use persistence sparingly, but that require good behavior even in pathological cases.

In [ ]:
data BootstrappedQueue a = E | Q Int [a] (BootstrappedQueue [a]) Int [a]

-- | Maintain the invariant that |lenr| <= |lenfm| and rotate if necessary.
checkQ :: Int -> [a] -> BootstrappedQueue [a] -> Int -> [a] -> BootstrappedQueue a
checkQ lenfm f m lenr r =
    if lenr <= lenfm then checkF lenfm f m lenr r
    else checkF (lenfm + lenr) f (snoc m (reverse r)) 0 []

-- | Maintain the invariant that the front is non-empty
checkF :: Int -> [a] -> BootstrappedQueue [a] -> Int -> [a] -> BootstrappedQueue a
checkF _ [] E _ _ = E
checkF lenfm [] m lenr r = Q lenfm (head m) (tail m) lenr r
checkF lenfm f m lenr r = Q lenfm f m lenr r

instance Queue BootstrappedQueue where
    
    -- | Constructs new queue in O(1)
    empty = E

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty E = True
    isEmpty _ = False
    
    -- | Appends new element to the back of the queue in O(log*(n)) amortized time.
    snoc E x = Q 1 [x] E 0 []
    snoc (Q lenfm f m lenr r) x = checkQ lenfm f m (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1) worst case time.
    head E = error "Queue is empty"
    head (Q _ (x:_) _ _ _) = x
    
    -- | Removes the head of the queue and returns the rest in O(log*(n)) amortized time.
    tail E = error "Queue is empty"
    tail (Q lenfm (x:f') m lenr r) = checkQ (lenfm - 1) f' m lenr r